In [2]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import gzip, json
from Bio import SeqIO
import scipy.sparse as sp
from collections import Counter
import numpy as np
import pandas as pd
fasta_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/reads.fasta.gz'
paf_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/alignment.paf.gz'
with gzip.open(paf_gz_file, "rt") as file:
    max_values = {}  
    for row in file:  
        columns = row.strip().split('\t') 
        query_id = columns[0]  
        match_bases = int(columns[9]) 
        max_values[query_id] = columns 
        if query_id in max_values:  
            if match_bases > int(max_values[query_id][9]):  
                max_values[match_bases] = columns
        else:  
            continue

In [5]:
from memory_profiler import profile
%load_ext memory_profiler


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 72.57 MiB, increment: 0.34 MiB


In [6]:
%memit x = 10+5

peak memory: 72.88 MiB, increment: 0.00 MiB


In [11]:
with gzip.open(fasta_gz_file, "rt") as handle:
    flag = 0
    seq_num = 0
    aligned_num = 0
    for record in SeqIO.parse(handle, "fasta"):
        seq_num += 1
        if record.id in max_values.keys():
            aligned_num+=1
            columns = max_values[record.id]
            if int(columns[9])/int(columns[1]) > 0.5:
                flag += 1
    print(flag/seq_num)
    print(aligned_num/seq_num)

0.9493518828919433
0.9829031176667784


In [ ]:
##输入需要有kmer_indice 这个字典，键是index，值是kmer的内容
class SimHash():

    @staticmethod
    def _hash(kmer: str, seed: int) -> int:
        hash_value = mmh3.hash(kmer, seed=seed)
        binary_string = format(hash_value & 0xFFFFFFFF, '032b')
        return binary_string

    @staticmethod
    def _get_read_hash_list(
        read_features: list,
        kmer_indice: Mapping(int, str), 
        repeats: int, 
        seed: int) -> Mapping(int,list):
        rng = np.random.default_rng(seed)
        hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeats, dtype=np.uint64,replace=False)
        repeat_all_read_hash = {}
        for k in range(repeats):
            s = hash_seeds[k]
            kmer_hash_indice={}
            all_read_hash = []
            for indice,kmer in kmer_indice:
                hash_value = SimHash._hash(kmer, seed=s)
                kmer_hash_indice[indice] = hash_value
            for features in read_features:
                read_hash = {}
                feature_count = dict(collections.Counter(features))
                for indice,count in feature_count.items():
                    hash_value = kmer_hash_indice[indice]
                    read_hash[hash_value] = count
                all_read_hash.append[read_hash]
        repeat_all_read_hash[k] = all_read_hash              
        return repeat_all_read_hash

    @staticmethod
    def get_simhash_value(hash_dict:dict):

        read_hash = []
        for hash_vaule,count in hash_dict:
            hash_list = list(hash_value)
            new_hash = [-1 if x == 0 else x for x in hash_list]
            weight_hash = count*new_hash
            read_hash.append(weight_hash)
        hash_arrary = np.arrary(read_hash)
        
                
            


            
        

    

In [8]:
from collections import Counter
colors = ['red', 'blue', 'red', 'green', 'blue', 'blue']
c = Counter(colors)
print (c)

Counter({'blue': 3, 'red': 2, 'green': 1})


In [6]:
import mmh3
hash_value = mmh3.hash('AAAATTTTGCATAACGGT',seed=10)
binary_string = format(hash_value & 0xFFFFFFFF, '032b')
binary_string

'00100101101010010101110010111001'

In [14]:
with gzip.open(paf_gz_file, "rt") as file:
    reads_aligned = []
    for row in file:  
        columns = row.strip().split('\t') 
        reads_aligned.append(columns[0])

with gzip.open(fasta_gz_file, "rt") as handle:  # Open gzipped file in text mode
    for record in SeqIO.parse(handle, "fasta"):
        if record.id in reads_aligned:
            print(record.id)

SRR25570561.4
SRR25570561.12
SRR25570561.17
SRR25570561.20
SRR25570561.31
SRR25570561.34
SRR25570561.41
SRR25570561.42
SRR25570561.44
SRR25570561.46
SRR25570561.52
SRR25570561.57
SRR25570561.61
SRR25570561.63
SRR25570561.73
SRR25570561.75
SRR25570561.86
SRR25570561.88
SRR25570561.89
SRR25570561.97
SRR25570561.99
SRR25570561.104
SRR25570561.105
SRR25570561.111
SRR25570561.112
SRR25570561.115
SRR25570561.118
SRR25570561.126
SRR25570561.128
SRR25570561.143
SRR25570561.147
SRR25570561.155
SRR25570561.160
SRR25570561.161
SRR25570561.167
SRR25570561.171
SRR25570561.172
SRR25570561.183
SRR25570561.198
SRR25570561.200
SRR25570561.201
SRR25570561.205
SRR25570561.209
SRR25570561.217
SRR25570561.219
SRR25570561.220
SRR25570561.221
SRR25570561.225
SRR25570561.231
SRR25570561.244
SRR25570561.245
SRR25570561.254
SRR25570561.258
SRR25570561.260
SRR25570561.263
SRR25570561.269
SRR25570561.273
SRR25570561.275
SRR25570561.289
SRR25570561.290
SRR25570561.294
SRR25570561.319
SRR25570561.322
SRR25570561.33